In [10]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

# Key Question : How does the demographic and the income level of the people affect the preference and acceptance of self-driving cars? 

## Part 1： Cleaning

In [11]:
#Read raw file
drive_selfcar_df=pd.read_csv('https://raw.githubusercontent.com/EvaW01/schulich_ai_fundamentals/main/MMAI_Fundamental_Assignment.csv')

drive_selfcar_df.head()


,Unnamed: 0,Question_key,Interview_start_time,Interview_end_time,Device_Type,Language,Form_No,Tablet_category,Read_about_SelfDrive_Car,Widespread_use_SelfDrive_Car,...,People_Political_Party,People_Political_Type,People_Political_Type_Category,Income_Level,Vote_Registered_Adress,Ideology,Interaction_Frequency,Volume_Sum,Income_Class,Weight_99_Percent
0,0,100260.0,2021-11-03 14:25:27,2021-11-03 14:45:34,Laptop/PC,English,Form 2,Non-tablet HH,NaN,NaN,...,NaN,Rep/Lean Rep,Conservative Rep/Lean,"$50,000 to less than $60,000",You are ABSOLUTELY CERTAIN that you are regist...,Very conservative,Several times a day,No,Middle income,0.206396
1,1,100314.0,2021-11-04 12:35:35,2021-11-04 12:55:29,Smartphone,English,Form 1,Non-tablet HH,A little,Not sure,...,NaN,Rep/Lean Rep,Moderate/Liberal Rep/Lean,"$40,000 to less than $50,000",You are ABSOLUTELY CERTAIN that you are regist...,Liberal,Several times a day,Yes,Middle income,0.315090
2,2,100363.0,2021-11-03 20:23:43,2021-11-03 20:36:24,Smartphone,English,Form 1,Non-tablet HH,A little,Good idea for society,...,NaN,Dem/Lean Dem,Moderate/Conservative Dem/Lean,"$100,000 or more",You are ABSOLUTELY CERTAIN that you are regist...,Moderate,Several times a day,No,Upper income,0.829579
3,3,100598.0,2021-11-02 13:01:05,2021-11-04 12:37:42,Laptop/PC,English,Form 2,Non-tablet HH,NaN,NaN,...,NaN,Rep/Lean Rep,Conservative Rep/Lean,"$100,000 or more",You are ABSOLUTELY CERTAIN that you are regist...,Conservative,Several times a day,Yes,Upper income,0.337527
4,4,100637.0,2021-11-02 12:32:58,2021-11-02 12:46:23,Laptop/PC,English,Form 2,Non-tablet HH,NaN,NaN,...,The Republican Party,Rep/Lean Rep,Conservative Rep/Lean,"$30,000 to less than $40,000",You are ABSOLUTELY CERTAIN that you are regist...,Very conservative,Less often,No,Lower income,1.210606


In [12]:
#drive_selfcar_df['Income_Level'] = drive_selfcar_df['Income_Level'].apply(lambda x: '50000 - 60000' if x == '$50,000 to less than $60,000')

drive_selfcar_df['Income_Level'] = drive_selfcar_df['Income_Level'].apply(
    lambda x: '50000 - 60000' if str(x).lower() == '$50,000 to less than $60,000'.lower() else x)
drive_selfcar_df['Income_Level'] = drive_selfcar_df['Income_Level'].apply(
    lambda x: '40000 - 50000' if str(x).lower() == '$40,000 to less than $50,000'.lower() else x)
drive_selfcar_df['Income_Level'] = drive_selfcar_df['Income_Level'].apply(
    lambda x: '100000' if str(x).lower() == '$100,000 or more'.lower() else x)
drive_selfcar_df['Income_Level'] = drive_selfcar_df['Income_Level'].apply(
    lambda x: '30000 - 40000' if str(x).lower() == '$30,000 to less than $40,000'.lower() else x)
drive_selfcar_df['Income_Level'] = drive_selfcar_df['Income_Level'].apply(
    lambda x: '30000' if str(x).lower() == 'Less than $30,000'.lower() else x)
drive_selfcar_df['Income_Level'] = drive_selfcar_df['Income_Level'].apply(
    lambda x: '70000 - 80000' if str(x).lower() == '$70,000 to less than $80,000'.lower() else x)
drive_selfcar_df['Income_Level'] = drive_selfcar_df['Income_Level'].apply(
    lambda x: '80000 - 90000' if str(x).lower() == '$80,000 to less than $90,000'.lower() else x)
drive_selfcar_df['Income_Level'] = drive_selfcar_df['Income_Level'].apply(
    lambda x: '90000 - 100000' if str(x).lower() == '$90,000 to less than $100,000'.lower() else x)
drive_selfcar_df['Income_Level'] = drive_selfcar_df['Income_Level'].apply(
    lambda x: '60000 - 70000' if str(x).lower() == '$60,000 to less than $70,000'.lower() else x)
drive_selfcar_df['Income_Level'] = drive_selfcar_df['Income_Level'].apply(
    lambda x: 0 if str(x).lower() == 'Refused'.lower() else x)

drive_selfcar_df['Income_Level'].unique()

array(['50000 - 60000', '40000 - 50000', '100000', '30000 - 40000',
       '30000', '70000 - 80000', 0, '80000 - 90000', '90000 - 100000',
       '60000 - 70000'], dtype=object)

In [13]:
drive_selfcar_df = drive_selfcar_df.drop(['Form_No', 'Tablet_category', 'Hispanic', 'Hispanic_Origin', 'Religion', 'Birth_Religion',
       'Religion_Category', 'Religion_Attend_Place', 'Religion_Importance',
       'Religion_Prayer', 'People_Political', 'People_Political_Party',
       'People_Political_Type', 'People_Political_Type_Category'], axis=1)

drive_selfcar_df

,Unnamed: 0,Question_key,Interview_start_time,Interview_end_time,Device_Type,Language,Read_about_SelfDrive_Car,Widespread_use_SelfDrive_Car,Choice_drive_SelfDrive_Car,Passenger_SelfDrive_Car_CaseA,...,Citizen,Birthplace,Marital_Status,Income_Level,Vote_Registered_Adress,Ideology,Interaction_Frequency,Volume_Sum,Income_Class,Weight_99_Percent
0,0,1.002600e+05,2021-11-03 14:25:27,2021-11-03 14:45:34,Laptop/PC,English,NaN,NaN,NaN,NaN,...,Yes,"U.S. – 50 states, District of Columbia",Never been married,50000 - 60000,You are ABSOLUTELY CERTAIN that you are regist...,Very conservative,Several times a day,No,Middle income,0.206396
1,1,1.003140e+05,2021-11-04 12:35:35,2021-11-04 12:55:29,Smartphone,English,A little,Not sure,Probably NOT want,Probably would happen,...,Yes,"U.S. – 50 states, District of Columbia",Divorced,40000 - 50000,You are ABSOLUTELY CERTAIN that you are regist...,Liberal,Several times a day,Yes,Middle income,0.315090
2,2,1.003630e+05,2021-11-03 20:23:43,2021-11-03 20:36:24,Smartphone,English,A little,Good idea for society,Probably want,Definitely would happen,...,Yes,"U.S. – 50 states, District of Columbia",Married,100000,You are ABSOLUTELY CERTAIN that you are regist...,Moderate,Several times a day,No,Upper income,0.829579
3,3,1.005980e+05,2021-11-02 13:01:05,2021-11-04 12:37:42,Laptop/PC,English,NaN,NaN,NaN,NaN,...,Yes,"U.S. – 50 states, District of Columbia",Married,100000,You are ABSOLUTELY CERTAIN that you are regist...,Conservative,Several times a day,Yes,Upper income,0.337527
4,4,1.006370e+05,2021-11-02 12:32:58,2021-11-02 12:46:23,Laptop/PC,English,NaN,NaN,NaN,NaN,...,Yes,"U.S. – 50 states, District of Columbia",Married,30000 - 40000,You are ABSOLUTELY CERTAIN that you are regist...,Very conservative,Less often,No,Lower income,1.210606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10255,10255,2.021010e+11,2021-11-02 15:06:47,2021-11-02 15:21:18,Smartphone,English,NaN,NaN,NaN,NaN,...,Yes,"U.S. – 50 states, District of Columbia",Married,30000,You are NOT registered to vote at your current...,Moderate,Almost constantly,No,Lower income,0.425901
10256,10256,2.021010e+11,2021-11-02 12:18:58,2021-11-02 12:32:49,Smartphone,English,NaN,NaN,NaN,NaN,...,Yes,"U.S. – 50 states, District of Columbia",Married,100000,You are ABSOLUTELY CERTAIN that you are regist...,Liberal,Several times a day,Yes,Upper income,0.157638
10257,10257,2.021010e+11,2021-11-04 16:30:29,2021-11-06 11:08:56,Smartphone,English,A lot,Good idea for society,Definitely want,Definitely would happen,...,Yes,"U.S. – 50 states, District of Columbia",Living with a partner,30000,You are ABSOLUTELY CERTAIN that you are regist...,Liberal,Almost constantly,Yes,Lower income,0.282079
10258,10258,2.021010e+11,2021-11-06 15:22:13,2021-11-06 16:10:36,Laptop/PC,English,A little,Good idea for society,Definitely want,Definitely would happen,...,Yes,"U.S. – 50 states, District of Columbia",Married,40000 - 50000,You are ABSOLUTELY CERTAIN that you are regist...,Moderate,Several times a day,No,Middle income,0.441525


In [14]:
drive_selfcar_df.dropna(inplace = True)

In [15]:
#'A man', 'A woman', 'In some other way', 'Refused'

drive_selfcar_df['Gender'] = drive_selfcar_df['Gender'].apply(
    lambda x: 'Man' if str(x).lower() == 'A man'.lower() else x)
drive_selfcar_df['Gender'] = drive_selfcar_df['Gender'].apply(
    lambda x: 'Woman' if str(x).lower() == 'A woman'.lower() else x)
drive_selfcar_df['Gender'] = drive_selfcar_df['Gender'].apply(
    lambda x: 'Other' if str(x).lower() == 'In some other way'.lower() else x)
drive_selfcar_df['Gender'] = drive_selfcar_df['Gender'].apply(
    lambda x: 'Other' if str(x).lower() == 'Refused'.lower() else x)

drive_selfcar_df['Gender'].unique()

array(['Man', 'Woman', 'Other'], dtype=object)

In [16]:
drive_selfcar_df['Age'] = drive_selfcar_df['Age'].apply(
    lambda x: 0 if str(x).lower() == 'Refused'.lower() else x)
drive_selfcar_df['Age'].unique()

array(['65+', '30-49', '50-64', '18-29', 0], dtype=object)

In [17]:
def society_income_influence(row):
    if row['Widespread_use_SelfDrive_Car'] == 'Good idea for society' and row['Accident_SelfDrive_Car'] == 'Increase the number of people killed or injured in traffic accidents':
        return 'Contrary Opinion'
    elif row['Widespread_use_SelfDrive_Car'] == 'Good idea for society' and row['Accident_SelfDrive_Car'] == 'Decrease the number of people killed or injured in traffic accidents':
        return 'Similar Opinion'
    elif row['Widespread_use_SelfDrive_Car'] == 'Bad idea for society' and row['Accident_SelfDrive_Car'] == 'Increase the number of people killed or injured in traffic accidents':
        return 'Similar Opinion'
    elif row['Widespread_use_SelfDrive_Car'] == 'Bad idea for society' and row['Accident_SelfDrive_Car'] == 'Decrease the number of people killed or injured in traffic accidents':
        return 'Contrary Opinion'
    else:
        return 'No Popular Opinion'

drive_selfcar_df['Preference_basedon_accident'] = drive_selfcar_df.apply(society_income_influence, axis=1)
drive_selfcar_df

,Unnamed: 0,Question_key,Interview_start_time,Interview_end_time,Device_Type,Language,Read_about_SelfDrive_Car,Widespread_use_SelfDrive_Car,Choice_drive_SelfDrive_Car,Passenger_SelfDrive_Car_CaseA,...,Birthplace,Marital_Status,Income_Level,Vote_Registered_Adress,Ideology,Interaction_Frequency,Volume_Sum,Income_Class,Weight_99_Percent,Preference_basedon_accident
1,1,1.003140e+05,2021-11-04 12:35:35,2021-11-04 12:55:29,Smartphone,English,A little,Not sure,Probably NOT want,Probably would happen,...,"U.S. – 50 states, District of Columbia",Divorced,40000 - 50000,You are ABSOLUTELY CERTAIN that you are regist...,Liberal,Several times a day,Yes,Middle income,0.315090,No Popular Opinion
2,2,1.003630e+05,2021-11-03 20:23:43,2021-11-03 20:36:24,Smartphone,English,A little,Good idea for society,Probably want,Definitely would happen,...,"U.S. – 50 states, District of Columbia",Married,100000,You are ABSOLUTELY CERTAIN that you are regist...,Moderate,Several times a day,No,Upper income,0.829579,Similar Opinion
5,5,1.008030e+05,2021-11-03 19:22:44,2021-11-03 19:41:04,Laptop/PC,English,A little,Bad idea for society,Definitely NOT want,Probably would NOT happen,...,"U.S. – 50 states, District of Columbia",Married,100000,You are ABSOLUTELY CERTAIN that you are regist...,Conservative,Several times a week,Yes,Upper income,0.302004,Similar Opinion
6,6,1.012240e+05,2021-11-03 18:18:21,2021-11-03 18:33:41,Smartphone,English,A little,Bad idea for society,Probably NOT want,Probably would NOT happen,...,"U.S. – 50 states, District of Columbia",Married,40000 - 50000,You are ABSOLUTELY CERTAIN that you are regist...,Conservative,About once a day,Yes,Middle income,0.287719,No Popular Opinion
8,8,1.014850e+05,2021-11-02 19:22:24,2021-11-02 19:35:23,Smartphone,English,A little,Bad idea for society,Probably NOT want,Probably would NOT happen,...,"U.S. – 50 states, District of Columbia",Widowed,100000,You are ABSOLUTELY CERTAIN that you are regist...,Conservative,Almost constantly,Yes,Upper income,0.276541,Similar Opinion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10252,10252,2.021010e+11,2021-11-03 18:50:06,2021-11-03 19:02:32,Laptop/PC,English,A little,Bad idea for society,Probably NOT want,Probably would happen,...,Another country,Married,30000,You are NOT registered to vote at your current...,Moderate,Almost constantly,Yes,Lower income,1.230882,Similar Opinion
10253,10253,2.021010e+11,2021-11-02 14:27:17,2021-11-02 15:01:43,Smartphone,English,A lot,Not sure,Definitely NOT want,Probably would happen,...,"U.S. – 50 states, District of Columbia",Married,50000 - 60000,You are NOT registered to vote at your current...,Refused,Several times a week,No,Middle income,8.524234,No Popular Opinion
10257,10257,2.021010e+11,2021-11-04 16:30:29,2021-11-06 11:08:56,Smartphone,English,A lot,Good idea for society,Definitely want,Definitely would happen,...,"U.S. – 50 states, District of Columbia",Living with a partner,30000,You are ABSOLUTELY CERTAIN that you are regist...,Liberal,Almost constantly,Yes,Lower income,0.282079,Contrary Opinion
10258,10258,2.021010e+11,2021-11-06 15:22:13,2021-11-06 16:10:36,Laptop/PC,English,A little,Good idea for society,Definitely want,Definitely would happen,...,"U.S. – 50 states, District of Columbia",Married,40000 - 50000,You are ABSOLUTELY CERTAIN that you are regist...,Moderate,Several times a day,No,Middle income,0.441525,Similar Opinion


In [18]:
drive_selfcar_df.columns

Index(['Unnamed: 0', 'Question_key', 'Interview_start_time',
       'Interview_end_time', 'Device_Type', 'Language',
       'Read_about_SelfDrive_Car', 'Widespread_use_SelfDrive_Car',
       'Choice_drive_SelfDrive_Car', 'Passenger_SelfDrive_Car_CaseA',
       'Passenger_SelfDrive_Car_CaseB', 'Passenger_SelfDrive_Car_CaseC',
       'Passenger_SelfDrive_Car_CaseD', 'Income_class_SelfDrive_Car',
       'Accident_SelfDrive_Car', 'UnnAccident_SelfDrive_Car',
       'Group_role_SelfDrive_Car_CaseA', 'Group_role_SelfDrive_Car_CaseB',
       'Group_role_SelfDrive_Car_CaseC', 'Group_role_SelfDrive_Car_CaseD',
       'Concern_SelfDrive_Car', 'Safety_SelfDrive_Car',
       'Accpt_circumstances_SelfDrive_Car_CaseA',
       'Accpt_circumstances_SelfDrive_Car_CaseB',
       'Accpt_circumstances_SelfDrive_Car_CaseC',
       'Accpt_circumstances_SelfDrive_Car_CaseD', 'Share_road_SelfDriveCar',
       'Purpose_SelfDriveCar_CaseA', 'Purpose_SelfDriveCar_CaseB',
       'Purpose_SelfDriveCar_CaseC', 'Pur

In [19]:
drive_selfcar_df['Accpt_circumstances_SelfDrive_Car_CaseA'].unique()

array(['More acceptable', 'No difference', 'Less acceptable', 'Refused'],
      dtype=object)

In [20]:
drive_selfcar_df['Ideology'].unique()

array(['Liberal', 'Moderate', 'Conservative', 'Very conservative',
       'Very liberal', 'Refused'], dtype=object)

In [21]:
drive_selfcar_df['Preference_basedon_accident'].unique()

array(['No Popular Opinion', 'Similar Opinion', 'Contrary Opinion'],
      dtype=object)

In [22]:
input_variables = ['Widespread_use_SelfDrive_Car',
       'Choice_drive_SelfDrive_Car',
       'Passenger_SelfDrive_Car_CaseB', 'Passenger_SelfDrive_Car_CaseC',
       'Accident_SelfDrive_Car', 
       'Accpt_circumstances_SelfDrive_Car_CaseA',
       'Accpt_circumstances_SelfDrive_Car_CaseB',
       'Accpt_circumstances_SelfDrive_Car_CaseC',
       'Accpt_circumstances_SelfDrive_Car_CaseD', 
       'Share_road_SelfDriveCar',
       'Purpose_SelfDriveCar_CaseA', 
       'Purpose_SelfDriveCar_CaseC',
       'Demographic_Region', 'Age', 'Gender', 'Education_Degree',
       'Ethnicity', 
       'Income_Level','Ideology']

In [23]:
selfvcar_inpt_df = drive_selfcar_df[input_variables]

In [24]:
selfvcar_inpt_df.head(10)

,Widespread_use_SelfDrive_Car,Choice_drive_SelfDrive_Car,Passenger_SelfDrive_Car_CaseB,Passenger_SelfDrive_Car_CaseC,Accident_SelfDrive_Car,Accpt_circumstances_SelfDrive_Car_CaseA,Accpt_circumstances_SelfDrive_Car_CaseB,Accpt_circumstances_SelfDrive_Car_CaseC,Accpt_circumstances_SelfDrive_Car_CaseD,Share_road_SelfDriveCar,Purpose_SelfDriveCar_CaseA,Purpose_SelfDriveCar_CaseC,Demographic_Region,Age,Gender,Education_Degree,Ethnicity,Income_Level,Ideology
1,Not sure,Probably NOT want,Probably would happen,Probably would NOT happen,Increase the number of people killed or injure...,More acceptable,More acceptable,No difference,More acceptable,Somewhat comfortable,Not sure,Favor,East North Central,65+,Man,"Some college, no degree",White non-Hispanic,40000 - 50000,Liberal
2,Good idea for society,Probably want,Definitely would happen,Probably would happen,Decrease the number of people killed or injure...,More acceptable,More acceptable,More acceptable,More acceptable,Very comfortable,Favor,Favor,Middle Atlantic,30-49,Woman,Postgraduate,White non-Hispanic,100000,Moderate
5,Bad idea for society,Definitely NOT want,Definitely would happen,Definitely would NOT happen,Increase the number of people killed or injure...,More acceptable,No difference,More acceptable,More acceptable,Not too comfortable,Favor,Oppose,West North Central,65+,Woman,College graduate/some post grad,White non-Hispanic,100000,Conservative
6,Bad idea for society,Probably NOT want,Probably would happen,Probably would NOT happen,Not make much difference,More acceptable,More acceptable,No difference,More acceptable,Not too comfortable,Oppose,Oppose,South Atlantic,65+,Man,"Some college, no degree",White non-Hispanic,40000 - 50000,Conservative
8,Bad idea for society,Probably NOT want,Probably would NOT happen,Definitely would NOT happen,Increase the number of people killed or injure...,More acceptable,No difference,More acceptable,No difference,Not too comfortable,Oppose,Oppose,Mountain,50-64,Woman,"Some college, no degree",Asian non-Hispanic,100000,Conservative
10,Good idea for society,Probably want,Probably would happen,Probably would NOT happen,Decrease the number of people killed or injure...,More acceptable,More acceptable,More acceptable,No difference,Very comfortable,Not sure,Not sure,Middle Atlantic,18-29,Woman,College graduate/some post grad,White non-Hispanic,100000,Conservative
11,Good idea for society,Definitely want,Probably would NOT happen,Definitely would happen,Decrease the number of people killed or injure...,More acceptable,No difference,More acceptable,More acceptable,Very comfortable,Favor,Not sure,South Atlantic,65+,Woman,Associate's degree,White non-Hispanic,0,Liberal
13,Good idea for society,Probably want,Probably would happen,Probably would happen,Decrease the number of people killed or injure...,No difference,More acceptable,More acceptable,No difference,Somewhat comfortable,Favor,Favor,South Atlantic,30-49,Woman,Postgraduate,White non-Hispanic,80000 - 90000,Liberal
15,Bad idea for society,Definitely NOT want,Probably would NOT happen,Probably would NOT happen,Refused,More acceptable,More acceptable,Less acceptable,More acceptable,Not too comfortable,Oppose,Oppose,Mountain,65+,Man,College graduate/some post grad,White non-Hispanic,100000,Moderate
16,Not sure,Probably NOT want,Definitely would happen,Probably would happen,Decrease the number of people killed or injure...,More acceptable,More acceptable,More acceptable,More acceptable,Somewhat comfortable,Not sure,Oppose,Middle Atlantic,65+,Woman,Postgraduate,White non-Hispanic,30000,Moderate


In [28]:
# get the score except the DCARS11

widespread_use_mapping = {
    'Good idea for society': 5,
    'Bad idea for society': 1,
    'Not sure': 3
}

choice_drive_mapping = {
    'Definitely NOT want': 1,
    'Probably NOT want': 2,
    'Probably want': 4,
    'Definitely want': 5
}

passenger_caseb_mapping = {
    'Definitely would happen': 1,
    'Probably would happen': 2,
    'Probably would NOT happen': 4,
    'Definitely would NOT happen': 5
}

passenger_casec_mapping = {
    'Definitely would happen': 5,
    'Probably would happen': 4,
    'Probably would NOT happen': 2,
    'Definitely would NOT happen': 1
}

accident_mapping = {
    'Increase the number of people killed or injured in traffic accidents': 1,
    'Decrease the number of people killed or injured in traffic accidents': 5,
    'Not make much difference': 3
}

share_road_mapping = {
    'Extremely comfortable': 5,
    'Very comfortable': 4,
    'Somewhat comfortable': 3,
    'Not too comfortable': 2,
    'Not comfortable at all': 1
}

purpose_casea_mapping = {
    'Favor': 5,
    'Oppose': 1,
    'Not sure': 3
}

purpose_casea_mapping = {
    'Favor': 5,
    'Oppose': 1,
    'Not sure': 3
}

purpose_casec_mapping = {
    'Favor': 5,
    'Oppose': 1,
    'Not sure': 3
}

selfvcar_inpt_df['Widespread_use_SelfDrive_Car_Score'] = selfvcar_inpt_df['Widespread_use_SelfDrive_Car'].map(widespread_use_mapping)
selfvcar_inpt_df['Choice_drive_SelfDrive_Car_Score'] = selfvcar_inpt_df['Choice_drive_SelfDrive_Car'].map(choice_drive_mapping)
selfvcar_inpt_df['Passenger_SelfDrive_Car_CaseB_Score'] = selfvcar_inpt_df['Passenger_SelfDrive_Car_CaseB'].map(passenger_caseb_mapping)
selfvcar_inpt_df['Passenger_SelfDrive_Car_CaseC_Score'] = selfvcar_inpt_df['Passenger_SelfDrive_Car_CaseC'].map(passenger_casec_mapping)
selfvcar_inpt_df['Accident_SelfDrive_Car_Score'] = selfvcar_inpt_df['Accident_SelfDrive_Car'].map(accident_mapping)
selfvcar_inpt_df['Share_road_SelfDriveCar_Score'] = selfvcar_inpt_df['Share_road_SelfDriveCar'].map(share_road_mapping)
selfvcar_inpt_df['Purpose_SelfDriveCar_CaseA_Score'] = selfvcar_inpt_df['Purpose_SelfDriveCar_CaseA'].map(purpose_casea_mapping)
selfvcar_inpt_df['Purpose_SelfDriveCar_CaseC_Score'] = selfvcar_inpt_df['Purpose_SelfDriveCar_CaseC'].map(purpose_casea_mapping)

selfvcar_inpt_df



/var/folders/dt/wzt2_nrs77vgrsp6wgvfx7d00000gn/T/ipykernel_51806/1228031357.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selfvcar_inpt_df['Widespread_use_SelfDrive_Car_Score'] = selfvcar_inpt_df['Widespread_use_SelfDrive_Car'].map(widespread_use_mapping)
/var/folders/dt/wzt2_nrs77vgrsp6wgvfx7d00000gn/T/ipykernel_51806/1228031357.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selfvcar_inpt_df['Choice_drive_SelfDrive_Car_Score'] = selfvcar_inpt_df['Choice_drive_SelfDrive_Car'].map(choice_drive_m

,Widespread_use_SelfDrive_Car,Choice_drive_SelfDrive_Car,Passenger_SelfDrive_Car_CaseB,Passenger_SelfDrive_Car_CaseC,Accident_SelfDrive_Car,Accpt_circumstances_SelfDrive_Car_CaseA,Accpt_circumstances_SelfDrive_Car_CaseB,Accpt_circumstances_SelfDrive_Car_CaseC,Accpt_circumstances_SelfDrive_Car_CaseD,Share_road_SelfDriveCar,...,Income_Level,Ideology,Widespread_use_SelfDrive_Car_Score,Choice_drive_SelfDrive_Car_Score,Passenger_SelfDrive_Car_CaseB_Score,Passenger_SelfDrive_Car_CaseC_Score,Accident_SelfDrive_Car_Score,Share_road_SelfDriveCar_Score,Purpose_SelfDriveCar_CaseA_Score,Purpose_SelfDriveCar_CaseC_Score
1,Not sure,Probably NOT want,Probably would happen,Probably would NOT happen,Increase the number of people killed or injure...,More acceptable,More acceptable,No difference,More acceptable,Somewhat comfortable,...,40000 - 50000,Liberal,3.0,2.0,2.0,2.0,1.0,3.0,3.0,5.0
2,Good idea for society,Probably want,Definitely would happen,Probably would happen,Decrease the number of people killed or injure...,More acceptable,More acceptable,More acceptable,More acceptable,Very comfortable,...,100000,Moderate,5.0,4.0,1.0,4.0,5.0,4.0,5.0,5.0
5,Bad idea for society,Definitely NOT want,Definitely would happen,Definitely would NOT happen,Increase the number of people killed or injure...,More acceptable,No difference,More acceptable,More acceptable,Not too comfortable,...,100000,Conservative,1.0,1.0,1.0,1.0,1.0,2.0,5.0,1.0
6,Bad idea for society,Probably NOT want,Probably would happen,Probably would NOT happen,Not make much difference,More acceptable,More acceptable,No difference,More acceptable,Not too comfortable,...,40000 - 50000,Conservative,1.0,2.0,2.0,2.0,3.0,2.0,1.0,1.0
8,Bad idea for society,Probably NOT want,Probably would NOT happen,Definitely would NOT happen,Increase the number of people killed or injure...,More acceptable,No difference,More acceptable,No difference,Not too comfortable,...,100000,Conservative,1.0,2.0,4.0,1.0,1.0,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10252,Bad idea for society,Probably NOT want,Definitely would happen,Probably would NOT happen,Increase the number of people killed or injure...,No difference,More acceptable,No difference,More acceptable,Somewhat comfortable,...,30000,Moderate,1.0,2.0,1.0,2.0,1.0,3.0,1.0,1.0
10253,Not sure,Definitely NOT want,Definitely would happen,Probably would happen,Decrease the number of people killed or injure...,More acceptable,More acceptable,More acceptable,More acceptable,Not too comfortable,...,50000 - 60000,Refused,3.0,1.0,1.0,4.0,5.0,2.0,3.0,5.0
10257,Good idea for society,Definitely want,Definitely would happen,Definitely would happen,Increase the number of people killed or injure...,More acceptable,More acceptable,More acceptable,More acceptable,Extremely comfortable,...,30000,Liberal,5.0,5.0,1.0,5.0,1.0,5.0,5.0,5.0
10258,Good idea for society,Definitely want,Probably would happen,Probably would happen,Decrease the number of people killed or injure...,More acceptable,More acceptable,No difference,No difference,Very comfortable,...,40000 - 50000,Moderate,5.0,5.0,2.0,4.0,5.0,4.0,5.0,1.0


In [30]:
# Try do the weight_avg

weights = {
    'Widespread_use_SelfDrive_Car_Score': 0.25,
    'Choice_drive_SelfDrive_Car_Score': 0.75 / 6,
    'Passenger_SelfDrive_Car_CaseB_Score': 0.75 / 6,
    'Passenger_SelfDrive_Car_CaseC_Score': 0.75 / 6,
    'Accident_SelfDrive_Car_Score': 0.75 / 6,
    'Share_road_SelfDriveCar_Score': 0.75 / 6,
    'Purpose_SelfDriveCar_CaseA_Score': 0.75 / 6
}

selfvcar_inpt_df['Acceptance_Score'] = (
    selfvcar_inpt_df['Widespread_use_SelfDrive_Car_Score'] * weights['Widespread_use_SelfDrive_Car_Score'] +
    selfvcar_inpt_df['Choice_drive_SelfDrive_Car_Score'] * weights['Choice_drive_SelfDrive_Car_Score'] +
    selfvcar_inpt_df['Passenger_SelfDrive_Car_CaseB_Score'] * weights['Passenger_SelfDrive_Car_CaseB_Score'] +
    selfvcar_inpt_df['Passenger_SelfDrive_Car_CaseC_Score'] * weights['Passenger_SelfDrive_Car_CaseC_Score'] +
    selfvcar_inpt_df['Accident_SelfDrive_Car_Score'] * weights['Accident_SelfDrive_Car_Score'] +
    selfvcar_inpt_df['Share_road_SelfDriveCar_Score'] * weights['Share_road_SelfDriveCar_Score'] +
    selfvcar_inpt_df['Purpose_SelfDriveCar_CaseA_Score'] * weights['Purpose_SelfDriveCar_CaseA_Score']
)

selfvcar_inpt_df

/var/folders/dt/wzt2_nrs77vgrsp6wgvfx7d00000gn/T/ipykernel_51806/1015794448.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selfvcar_inpt_df['Acceptance_Score'] = (


,Widespread_use_SelfDrive_Car,Choice_drive_SelfDrive_Car,Passenger_SelfDrive_Car_CaseB,Passenger_SelfDrive_Car_CaseC,Accident_SelfDrive_Car,Accpt_circumstances_SelfDrive_Car_CaseA,Accpt_circumstances_SelfDrive_Car_CaseB,Accpt_circumstances_SelfDrive_Car_CaseC,Accpt_circumstances_SelfDrive_Car_CaseD,Share_road_SelfDriveCar,...,Ideology,Widespread_use_SelfDrive_Car_Score,Choice_drive_SelfDrive_Car_Score,Passenger_SelfDrive_Car_CaseB_Score,Passenger_SelfDrive_Car_CaseC_Score,Accident_SelfDrive_Car_Score,Share_road_SelfDriveCar_Score,Purpose_SelfDriveCar_CaseA_Score,Purpose_SelfDriveCar_CaseC_Score,Acceptance_Score
1,Not sure,Probably NOT want,Probably would happen,Probably would NOT happen,Increase the number of people killed or injure...,More acceptable,More acceptable,No difference,More acceptable,Somewhat comfortable,...,Liberal,3.0,2.0,2.0,2.0,1.0,3.0,3.0,5.0,2.375
2,Good idea for society,Probably want,Definitely would happen,Probably would happen,Decrease the number of people killed or injure...,More acceptable,More acceptable,More acceptable,More acceptable,Very comfortable,...,Moderate,5.0,4.0,1.0,4.0,5.0,4.0,5.0,5.0,4.125
5,Bad idea for society,Definitely NOT want,Definitely would happen,Definitely would NOT happen,Increase the number of people killed or injure...,More acceptable,No difference,More acceptable,More acceptable,Not too comfortable,...,Conservative,1.0,1.0,1.0,1.0,1.0,2.0,5.0,1.0,1.625
6,Bad idea for society,Probably NOT want,Probably would happen,Probably would NOT happen,Not make much difference,More acceptable,More acceptable,No difference,More acceptable,Not too comfortable,...,Conservative,1.0,2.0,2.0,2.0,3.0,2.0,1.0,1.0,1.750
8,Bad idea for society,Probably NOT want,Probably would NOT happen,Definitely would NOT happen,Increase the number of people killed or injure...,More acceptable,No difference,More acceptable,No difference,Not too comfortable,...,Conservative,1.0,2.0,4.0,1.0,1.0,2.0,1.0,1.0,1.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10252,Bad idea for society,Probably NOT want,Definitely would happen,Probably would NOT happen,Increase the number of people killed or injure...,No difference,More acceptable,No difference,More acceptable,Somewhat comfortable,...,Moderate,1.0,2.0,1.0,2.0,1.0,3.0,1.0,1.0,1.500
10253,Not sure,Definitely NOT want,Definitely would happen,Probably would happen,Decrease the number of people killed or injure...,More acceptable,More acceptable,More acceptable,More acceptable,Not too comfortable,...,Refused,3.0,1.0,1.0,4.0,5.0,2.0,3.0,5.0,2.750
10257,Good idea for society,Definitely want,Definitely would happen,Definitely would happen,Increase the number of people killed or injure...,More acceptable,More acceptable,More acceptable,More acceptable,Extremely comfortable,...,Liberal,5.0,5.0,1.0,5.0,1.0,5.0,5.0,5.0,4.000
10258,Good idea for society,Definitely want,Probably would happen,Probably would happen,Decrease the number of people killed or injure...,More acceptable,More acceptable,No difference,No difference,Very comfortable,...,Moderate,5.0,5.0,2.0,4.0,5.0,4.0,5.0,1.0,4.375
